# GPyTorch

GPyTorch is a PyTorch-based library designed for implementing Gaussian processes. It was introduced by Jacob R. Gardner, Geoff Pleiss, David Bindel, Kilian Q. Weinberger and Andrew Gordon Wilson – researchers at Cornel University (research paper).

Before going into the details of GPyTorch, let us first understand what a Gaussian process means, in short.

To read more about this, you can refer to [this](https://analyticsindiamag.com/guide-to-gpytorch-a-python-library-for-gaussian-process-models/) article.

# Practical implementation

Here’s a demonstration of training an RBF kernel Gaussian process on the following function:

y = sin(2x) + E             …(i)

E ~ (0, 0.04)

 (where 0 is mean of the normal distribution and 0.04 is the variance)

Install the GPyTorch library

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels tensorflow keras --user -q

In [ ]:
!python -m pip install gpytorch --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

Import required libraries

In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
%matplotlib inline #for visualization plots to appear at the frontend 

Prepare training data

In [ ]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

Define the GP model

We have used exact inference – the simplest form of GP model

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

For most of the GP regression models, following objects should be constructed:

> * A ‘GP Model’ which handles most of the inference
> * A ‘Likelihood’
> * A ‘Mean’ defining prior mean of GP
> * A ‘Kernel’ defining covariance of GP
> * A Multivariate Normal Distribution

The two methods defined above are components of the Exact (non-variational) GP model.

The _init_ method takes a likelihood and the training data. It then constructs objects like mean module and kernel module required for the ‘forward’ method of the model. The ‘forward’ method takes in some data x. It returns a multivariate normal distribution with prior mean and covariance computed at x.

  Initialize likelihood

In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()

Initialize the GP model

In [ ]:
model = ExactGPModel(train_x, train_y, likelihood)

  Find optimal hyperparameters of the model

In [ ]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

Use Adam optimization algorithm

In [ ]:
# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

  Define loss for GP

In [ ]:
# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

Compute loss, length scale (i.e. length of twists and turns in the function) and noise for each iteration of the GP

In [ ]:
for i in range(20):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iteration %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, 20, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

  Make prediction with the model

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

  Make predictions by feeding model through likelihood

In [ ]:
# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0, 1, 51)
    observed_pred = likelihood(model(test_x))

Plot the fitted model

In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(8, 8))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])